In [2]:
import numpy as np
import os
import pandas as pd

## Data Preprocessing

In [3]:
def fen_to_array(fen_string):
  # map for each piece
  piece_to_int = {
    'r': 1, 'n': 2, 'b': 3, 'q': 4, 'k': 5, 'p': 6,
    'R': 7, 'N': 8, 'B': 9, 'Q': 10, 'K': 11, 'P': 12,
    '.': 0  # empty square
  }

  # Split the FEN string at spaces - the first part represents the board, the rest is special notation
  board_part = fen_string.split(' ')[0]

  # Replacing numbers for their corresponding number of empty squares
  for digit in '12345678':
    board_part = board_part.replace(digit, '.' * int(digit))

  # Replacing slashes with newlines to represent rows
  board_part = board_part.replace('/', '\n')

  # Convert the string to an 8x8 matrix which represents the chess board 
  board = [list(row) for row in board_part.split('\n')]

  # Convert board to a 3D array (8x8x12) where each layer represents a different piece type
  board_array = [[[0 for _ in range(12)] for _ in range(8)] for _ in range(8)]

  for i, row in enumerate(board):
    for j, piece in enumerate(row):
      if piece != '.':
        board_array[i][j][piece_to_int[piece] - 1] = 1

  return board_array

In [23]:
data = pd.read_csv('../chess_evaluations.csv')
data

,FEN,Evaluation
0,rnbqkbnr/pppppppp/8/8/3P4/8/PPP1PPPP/RNBQKBNR ...,31
1,rnbqkb1r/pppppppp/5n2/8/3P4/8/PPP1PPPP/RNBQKBN...,33
2,rnbqkb1r/pppppppp/5n2/8/3P4/5N2/PPP1PPPP/RNBQK...,24
3,rnbqkb1r/ppp1pppp/5n2/3p4/3P4/5N2/PPP1PPPP/RNB...,21
4,rnbqkb1r/ppp1pppp/5n2/3p4/3P4/4PN2/PPP2PPP/RNB...,12
...,...,...
460696,4Q3/5p2/3P2k1/1p3bp1/7p/2q4P/2p2PP1/4R1K1 w - ...,0
460697,6Q1/5p2/3P2k1/1p3bp1/7p/2q4P/2p2PP1/4R1K1 b - ...,0
460698,6Q1/5p2/3P3k/1p3bp1/7p/2q4P/2p2PP1/4R1K1 w - -...,0
460699,5Q2/5p2/3P3k/1p3bp1/7p/2q4P/2p2PP1/4R1K1 b - -...,0


In [24]:
data['FEN'] = data['FEN'].apply(fen_to_array)
data['FEN']

0         [[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1,...
1         [[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1,...
2         [[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1,...
3         [[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1,...
4         [[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1,...
                                ...                        
460696    [[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0,...
460697    [[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0,...
460698    [[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0,...
460699    [[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0,...
460700    [[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0,...
Name: FEN, Length: 460701, dtype: object

In [25]:
data.head()

,FEN,Evaluation
0,"[[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1,...",31
1,"[[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1,...",33
2,"[[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1,...",24
3,"[[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1,...",21
4,"[[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1,...",12


## ResNet Implementation 

In [3]:
import tensorflow as tf
from functools import partial

In [5]:
DefaultConv2D = partial(tf.keras.layers.Conv3D, kernel_size=(3, 3), strides=1, padding="same", kernel_initializer="he_normal", use_bias=False)

class ResidualUnit(tf.keras.layers.Layer):
  def __init__(self, filters, strides=1, activation="relu", **kwargs):
    super.__init__(**kwargs)
    self.activation = tf.keras.activations.get(activation)
    self.main_layers = [
      DefaultConv2D(filters=filters, strides=strides),
      tf.keras.layers.BatchNormalization(),
      self.activation,
      DefaultConv2D(filters=filters),
      tf.keras.layers.BatchNormalization()
    ]
    self.skip_layers = []
    if strides > 1:
      self.skip_layers = [
        DefaultConv2D(filters=filter, kernel_size=1, strides=strides),
        tf.keras.layers.BatchNormalization()
      ]
      
  def call(self, inputs):
    Z = inputs
    for layer in self.main_layers:
      Z = layer(Z)
    skip_Z = inputs
    for layer in self.skip_layers:
      skip_Z = layer(skip_Z)
    return self.activation(Z + skip_Z)
  

In [ ]:
model = tf.keras.Sequential([
  DefaultConv2D(64, kernel_size=7, strides=2, input_shape=[8, 8, 12]),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Activation("relu"),
  tf.keras.layers.MaxPool3D(pool_size=3, strides=2, padding="same"),
])
prev_filters=64
for filters in [64] * 3 + [128] * 4 + [256] * 6 + [512] * 3:
  strides = 1 if filters == prev_filters else 2
  model.add(ResidualUnit(filters, strides=strides))
  prev_filters = filters

model.add(tf.keras.layers.GlobalAveragePooling3D())
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(10, activation="softmax"))
